## 作业四
    根据特征预测语音是谁说的
目标:
   - 知道如何使用transfromer
   - 知道如何调试transformer参数
   - 学会构建[conformer](https://arxiv.org/abs/2005.08100)

## 下载数据

In [1]:
"""
    For Google drive, You can download data form any link below.
    If a link fails, please use another one.
"""
""" Download link 1 of Google drive """
# !gdown --id '1T0RPnu-Sg5eIPwQPfYysipfcz81MnsYe' --output Dataset.zip
""" Download link 2 of Google drive """
# !gdown --id '1CtHZhJ-mTpNsO-MqvAPIi4Yrt3oSBXYV' --output Dataset.zip
""" Download link 3 of Google drive """
# !gdown --id '14hmoMgB1fe6v50biIceKyndyeYABGrRq' --output Dataset.zip
""" Download link 4 of Google drive """
# !gdown --id '1e9x-Pjl3n7-9tK9LS_WjiMo2lru4UBH9' --output Dataset.zip
""" Download link 5 of Google drive """
# !gdown --id '10TC0g46bcAz_jkiMl65zNmwttT4RiRgY' --output Dataset.zip
""" Download link 6 of Google drive """
# !gdown --id '1MUGBvG_JjqO0C2JYHuyV3B0lvaf1kWIm' --output Dataset.zip
""" Download link 7 of Google drive """
# !gdown --id '18M91P5DHwILNyOlssZ57AiPOR0OwutOM' --output Dataset.zip
""" For Google drive, you can unzip the data by the command below. """
# !unzip Dataset.zip

"""
    For Dropbox, we split dataset into five files. 
    Please download all of them.
"""
# If Dropbox is not work. Please use google drive.
# !wget https://www.dropbox.com/s/vw324newiku0sz0/Dataset.tar.gz.aa?dl=0
# !wget https://www.dropbox.com/s/z840g69e7lnkayo/Dataset.tar.gz.ab?dl=0
# !wget https://www.dropbox.com/s/hl081e1ggonio81/Dataset.tar.gz.ac?dl=0
# !wget https://www.dropbox.com/s/fh3zd8ow668c4th/Dataset.tar.gz.ad?dl=0
# !wget https://www.dropbox.com/s/ydzygoy2pv6gw9d/Dataset.tar.gz.ae?dl=0
# !cat Dataset.tar.gz.* | tar zxvf -

"""
    For Onedrive, we split dataset into five files. 
    Please download all of them.
"""
!wget --no-check-certificate "https://onedrive.live.com/download?cid=10C95EE5FD151BFB&resid=10C95EE5FD151BFB%21106&authkey=ACB6opQR3CG9kmc" -O Dataset.tar.gz.aa
!wget --no-check-certificate "https://onedrive.live.com/download?cid=93DDDDD552E145DB&resid=93DDDDD552E145DB%21106&authkey=AP6EepjxSdvyV6Y" -O Dataset.tar.gz.ab
!wget --no-check-certificate "https://onedrive.live.com/download?cid=644545816461BCCC&resid=644545816461BCCC%21106&authkey=ALiefB0kI7Epb0Q" -O Dataset.tar.gz.ac
!wget --no-check-certificate "https://onedrive.live.com/download?cid=77CEBB3C3C512821&resid=77CEBB3C3C512821%21106&authkey=AAXCx4TTDYC0yjM" -O Dataset.tar.gz.ad
!wget --no-check-certificate "https://onedrive.live.com/download?cid=383D0E0146A11B02&resid=383D0E0146A11B02%21106&authkey=ALwVc4StVbig6QI" -O Dataset.tar.gz.ae
!cat Dataset.tar.gz.* | tar zxvf -

SyntaxError: invalid syntax (<ipython-input-1-6edb8f5c7ace>, line 38)

## 数据
    - 随机选择了600位说话者
    - 然后将原始波形预处理为mel谱图(梅尔光谱图:mel-spectrograms)

- 参数解析:
  - data_dir: 文件目录地址
  - metadata_path: metadata数据的地址.
  - segment_len: The length of audio segment for training. <br>
  
- The architecture of data directory:
    - data directory <br>
    |---- metadata.json <br>
    |---- testdata.json <br>
    |---- mapping.json <br>
    |---- uttr-{random string}.pt <br><br>

- metadata数据内容:
  - "n_mels": 梅尔光谱的场地.
  - "speakers":  A dictionary of speaker. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"

为了提高提高效率、在训练步骤中，我们将mel光谱图分割成若干段。

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


class MyDataset(Dataset):
    def __init__(self,data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker name to their corresponding(对应的) id. 
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]
        
        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))
    
        # Segmemt mel-spectrogram into "segment_len" frames.
        # 为了匹配读到数据长度不一样
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker
 
    def get_speaker_number(self):
        return self.speaker_num

## DataLoader

- 切分dataset中的数据、train:valid = 9:1


In [ ]:
def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()

def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = MyDataset(data_dir)
    speaker_num = dataset.get_speaker_number()

    # 切分训练数据和验证数据
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    # 训练数据加载器
    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    # 验证数据加载器
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    return train_loader, valid_loader, speaker_num


## 模型
- TransformerEncoderLayer():
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required). **输入输出的维度。**

    - nhead: the number of heads of the multiheadattention models (required). **多头注意力机制的Head数量。**

    - dim_feedforward: the dimension of the feedforward network model (default=2048). **前馈网络模型的维数**

    - dropout: the dropout value (default=0.1). **对两个子层的输出进行处理的dropout值。默认值：0.1。**

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).  **前馈神经网络的激活函数。默认值：relu。**

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers. ** TransformerEncoder是由N个transformer编码器层组成的堆栈**
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [ ]:
import torch.nn as nn
import torch.nn.functional as functional

class MyModule(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
        self.encoder_layer = nn.TransformerEncoderLayer(
        d_model=d_model, dim_feedforward=256, nhead=2
        )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
        nn.Linear(d_model, d_model),
        nn.ReLU(),
        nn.Linear(d_model, n_spks),
        )

    def forward(self, mels):
        """
        args:
        mels: (batch size, length, 40)
        return:
        out: (batch size, n_spks)
        """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        # mean pooling
        stats = out.mean(dim=1)

        # out: (batch, n_spks)
        out = self.pred_layer(stats)
        return out

## 自适应学习率(Learning rate schedule)
学习率时间表旨在通过根据预定义的时间表降低学习率来调整训练期间的学习率,与其对应的是固定学习率(Constant Learning Rate)<br>
对于transformer架构，学习率计划的设计不同于CNN。,以前的工作表明，学习率的预热对于使用transformer架构的训练模型是有用的。
- 学习率预热(Learning rate warmup):
  - 训练开始时learning rate 设置为0.
  - 在热身期间，学习率从0线性增加到初始学习率。

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(optimizer: Optimizer, num_warmup_steps: int, num_training_steps: int,
  num_cycles: float = 0.5,last_epoch: int = -1,):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
    return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

  return LambdaLR(optimizer, lr_lambda, last_epoch)


## 验证
验证模型的预测准确度

In [ ]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""
  
  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      mels, labels = batch
      mels = mels.to(device)
      labels = labels.to(device)
      pred = model(mels)
      loss = criterion(pred,labels).item()
      # Get the speaker id with highest probability.
      preds = pred.argmax(1)
      # Compute accuracy.
      # accuracy = (preds.argmax(dim=-1) == labels).float().mean()
      try:
        accuracy = torch.mean((preds == labels).float()).item()
      except(AttributeError):
        print( '报错信息:'.format(AttributeError))
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix({'running_loss':running_loss,"running_accuracy ":running_accuracy})

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


## 开始训练样本

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt


config = {
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }
loss_record = []
valid_record = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[Info]: Use {device} now!")
train_loader, valid_loader, speaker_num = get_dataloader(config['data_dir'], config['batch_size'], config['n_workers'])
train_iterator = iter(train_loader)
print("[Info]: Finish loading data!",flush = True)

# 加载模型
model = MyModule(n_spks=speaker_num).to(device)
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = AdamW(model.parameters(), lr=1e-3)
# 动态学习率计划
scheduler = get_cosine_schedule_with_warmup(optimizer, config['warmup_steps'], config['total_steps'])
print("[Info]: Finish creating model!",flush = True)


best_accuracy = -1.0
best_state_dict = None
valid_steps = config['valid_steps']
#进度条
pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

for step in range(config['total_steps']):
    # Get data
    try:
        batch = next(train_iterator)
    except StopIteration:
        train_iterator = iter(train_loader)
        batch = next(train_iterator)

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)
    pred = model(mels)
    loss = criterion(pred,labels)
    batch_loss = loss.item()
    # Get the speaker id with highest probability.
    preds = pred.argmax(1)
    # Compute accuracy.
    batch_accuracy = torch.mean((preds == labels).float()).item()

    # 更新模型参数‘    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    loss_record.append(batch_loss)
    # Log
    pbar.update()
    pbar.set_postfix({'batch_loss':batch_loss, 'batch_accuracy':batch_accuracy})

    # 做验证
    if (step + 1) % valid_steps == 0:
        pbar.close()

        valid_accuracy = valid(valid_loader, model, criterion, device)
        valid_record.append(valid_accuracy)
        # keep the best model
        if valid_accuracy > best_accuracy:
            best_accuracy = valid_accuracy
            best_state_dict = model.state_dict()
        pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % config['save_steps'] == 0 and best_state_dict is not None:
        torch.save(best_state_dict, config['save_path'])
        pbar.write("Step {step + 1}, best model saved. (accuracy={best_accuracy:%s})".format(best_accuracy))

pbar.close()

plt.figure(figsize=(6,4))
plt.plot(range(len(loss_record)), loss_record, c='tab:red', label='train')
plt.plot(range(len(valid_record)), valid_record, c='tab:cyan', label='valid')
plt.ylim(0.0, 5)
plt.ylabel('trian steps')
plt.xlabel('cross loss')
plt.title('Learning curve of train')
plt.legend()
plt.show()